In [21]:
import pandas as pd
import numpy as np
import torch

In [22]:
train = pd.read_csv('train.csv')   
test = pd.read_csv('test.csv')

In [23]:
y = train['avgPrice']
x = train.drop(['avgPrice'], axis=1)

In [24]:
x.year = [int(str(date)[4:6]) for date in x.year]
test.year = [int(str(date)[4:6]) for date in test.year]

In [25]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x = sc.fit_transform(x)
test = sc.transform(test)

In [26]:
x.shape

(2557, 5)

In [27]:
x_tensor = torch.FloatTensor(x)
y_tensor = torch.FloatTensor(y)
test_tensor = torch.FloatTensor(test)

In [28]:
layer1 = torch.nn.Linear(x.shape[1], 16, bias=True)
layer2 = torch.nn.Linear(16, 1, bias=True)
sigmoid = torch.nn.Sigmoid()

model = torch.nn.Sequential(layer1, sigmoid, layer2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [29]:
epochs = 15

for epoch in range(epochs):
    optimizer.zero_grad()
    hypothesis = model(x_tensor)
    cost = torch.nn.functional.mse_loss(hypothesis, y_tensor)
    cost.backward()
    optimizer.step()
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, epochs, cost.item()))

Epoch    0/15 Cost: 12015675.000000
Epoch    1/15 Cost: 10274587.000000
Epoch    2/15 Cost: 6987884.500000
Epoch    3/15 Cost: 4348667.000000
Epoch    4/15 Cost: 3031039.000000
Epoch    5/15 Cost: 2386502.000000
Epoch    6/15 Cost: 2084217.000000
Epoch    7/15 Cost: 1946406.625000
Epoch    8/15 Cost: 1882070.750000
Epoch    9/15 Cost: 1851555.875000
Epoch   10/15 Cost: 1836903.375000
Epoch   11/15 Cost: 1829954.000000
Epoch   12/15 Cost: 1826619.375000
Epoch   13/15 Cost: 1824924.250000
Epoch   14/15 Cost: 1824013.000000


/var/folders/wr/bqxxn9ss13b7snt4sxqbl0zm0000gn/T/ipykernel_54489/780144082.py:6: UserWarning: Using a target size (torch.Size([2557])) that is different to the input size (torch.Size([2557, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  cost = torch.nn.functional.mse_loss(hypothesis, y_tensor)


In [30]:
predict = model(test_tensor)
submit = pd.read_csv('submit_sample.csv')
submit['Expected'] = predict.detach()
submit.to_csv('submit.csv', index=False)